In [1]:
import praw
from psaw import PushshiftAPI
import datetime as dt
from dotenv import load_dotenv
import os
import re

In [2]:
load_dotenv()
client_id = os.getenv("CLIENTID")
client_secret = os.getenv("CLIENTSECRET")
user_agent = "AskHistGen:v1.0 (by u/AverageAngryPeasant)"

reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
api = PushshiftAPI(reddit)
data = list(api.search_submissions(subreddit='askhistorians', filter=['url','author', 'title', 'subreddit']))

Version 7.0.0 of praw is outdated. Version 7.1.2 was released 1 day ago.


In [3]:
removed = ["removed your question",
        "remove your submission",
        "remove your question",
        "remove your submission",
        "question has been removed",
        "submission has been removed",
        "question was removed",
        "submission was removed",
        "Sorry, we don't allow",
        "question is fine",
        "submission is fine",
        "meets our standard",
        "meet our standard",
        "[removed]"]

In [45]:
questions, answers = [], []
for i in range(len(data)):
    if data[i].link_flair_text and "question" not in data[i].link_flair_text.lower():
        continue
    if data[i].author and data[i].author.name == "AutoModerator":
        continue
        
    comments = [c for c in data[i].comments]
    if comments:
        while type(comments[-1]) == praw.models.reddit.more.MoreComments:
            comments = comments[:-1] + comments[-1].comments()
    comments = [c.body if not c.author or c.author.name != "AutoModerator" else None for c in comments]
    comments = list(filter(lambda x: x and len(x) > 500 and not any([r in x for r in removed]), comments))
    if comments:
        answers += comments
        questions.append(data[i].title)

In [46]:
for i in range(len(answers)):
    ans = answers[i]
    ans = re.sub(r'\(http\S+', '', ans)
    ans = re.sub(r'\[http\S+', '', ans)
    ans = re.sub("\([0-9]*\)", "", ans)
    ans = ans.replace("\\n\\n", " ").replace("&#x200B; ", "")
    ans = ans.translate(ans.maketrans("", "", "[]\\*>"))
    ans = " ".join(ans.split())
    answers[i] = ans

['What we group together as the "nobility" in Hungary actually changed throughout medieval and early modern Hungary. In the beginning, nobles were the leading office-holders around the ruler. This meaning overlapped after the mid-twelfth century with the idea that a nobleman is one whose ancestry is noble. After the fourteenth century, though, nobility really meant landowners. Owning land meant possessing "free landholdings" and "based on these landholdings, the nobility was beneficiary of the common law and of political freedom." (Quote from Akos Timon, Magyar alkotmany es jogtortenet, p. 122; For an English source, see Martyn Rady, Nobility, Land and Service in medieval Hungary, pp. 1-6). As nobility status became dependent on landowning, inheritance was one way to make new nobles quickly. By the fourteenth century, Hungarian nobles used to divide their lands between their sons, which led to some sons inheriting 12 households, half villages, a few mills or so. Doing so increased the 

In [47]:
with open("questions.txt", "w") as f:
    for item in question:
        f.write(item + "\n")

with open("answers.txt", "w") as f:
    for item in answers:
        f.write(item + "\n")

SyntaxError: invalid syntax (<ipython-input-47-eca942562efa>, line 3)